In [39]:
type BoxedStm = Box<Stm>;
type BoxedExp = Box<Exp>;
type BoxedExpList = Box<ExpList>;

enum Stm {
    Compound(BoxedStm, BoxedStm),
    Assign { id: String, exp: BoxedExp },
    Print(BoxedExpList),
}
impl Stm {
    fn boxed_compound(s1: BoxedStm, s2: BoxedStm) -> BoxedStm {
        Box::new(Stm::Compound(s1, s2))
    }

    fn boxed_assign(id: String, exp: BoxedExp) -> BoxedStm {
        Box::new(Stm::Assign { id, exp })
    }

    fn boxed_print(expList: BoxedExpList) -> BoxedStm {
        Box::new(Stm::Print(expList))
    }
}

enum BinOp {
    Plus,
    Minus,
    Times,
    Div,
}

enum Exp {
    Id(String),
    Num(i32),
    Op {
        left: BoxedExp,
        op: BinOp,
        right: BoxedExp,
    },
    Eseq(BoxedStm, BoxedExp),
}

impl Exp {
    fn boxed_id(id: String) -> BoxedExp {
        Box::new(Exp::Id(id))
    }

    fn boxed_num(num: i32) -> BoxedExp {
        Box::new(Exp::Num(num))
    }

    fn boxed_op(left: BoxedExp, op: BinOp, right: BoxedExp) -> BoxedExp {
        Box::new(Exp::Op { left, op, right })
    }

    fn boxed_eseq(stm: BoxedStm, exp: BoxedExp) -> BoxedExp {
        Box::new(Exp::Eseq(stm, exp))
    }
}

enum ExpList {
    Pair { head: BoxedExp, tail: BoxedExpList },
    Last(BoxedExp),
}

impl ExpList {
    fn boxed_pair(head: BoxedExp, tail: BoxedExpList) -> BoxedExpList {
        Box::new(ExpList::Pair { head, tail })
    }
    fn boxed_last(exp: BoxedExp) -> BoxedExpList {
        Box::new(ExpList::Last(exp))
    }
}

// a := 5 + 3 ; b := ( print ( a , a - 1) , 10 * a) ; print ( b )
let prog = Stm::boxed_compound(
    Stm::boxed_assign(
        "a".to_owned(),
        Exp::boxed_op(Exp::boxed_num(5), BinOp::Plus, Exp::boxed_num(3)),
    ),
    Stm::boxed_compound(
        Stm::boxed_assign(
            "b".to_owned(),
            Exp::boxed_eseq(
                Stm::boxed_print(ExpList::boxed_pair(
                    Exp::boxed_id("a".to_owned()),
                    ExpList::boxed_last(Exp::boxed_op(
                        Exp::boxed_id("a".to_owned()),
                        BinOp::Minus,
                        Exp::boxed_num(1),
                    )),
                )),
                Exp::boxed_op(
                    Exp::boxed_num(10),
                    BinOp::Times,
                    Exp::boxed_id("a".to_owned()),
                ),
            ),
        ),
        Stm::boxed_print(ExpList::boxed_last(Exp::boxed_id("b".to_owned()))),
    ),
);


（1） 写一个函数 int maxargs(A_stm)，告知给定语句中任意子表达式内的 print 语句的参数个数。例如，maxargs(prog) 的值是 2。

In [28]:
fn maxargs(stm: BoxedStm) -> usize {
    match *stm {
        Stm::Compound(stm1, stm2) => maxargs(stm1).max(maxargs(stm2)),
        Stm::Assign { exp, .. } => max_exp_args(exp),
        Stm::Print(expList) => count_exp_list(expList),
    }
}

fn max_exp_args(exp: BoxedExp) -> usize {
    match *exp {
        Exp::Op { left, right, .. } => max_exp_args(left).max(max_exp_args(right)),
        Exp::Eseq(stm, exp) => maxargs(stm).max(max_exp_args(exp)),
        _ => 0,
    }
}

fn count_exp_list(expList: BoxedExpList) -> usize {
    match *expList {
        ExpList::Pair { head, tail } => max_exp_args(head).max(1 + count_exp_list(tail)),
        ExpList::Last(exp) => 1 + max_exp_args(exp),
    }
}


println!("print maxargs is {:?}.", maxargs(prog));


print maxargs is 2.


（2） 写一个函数 void interp(A_stm), 对一个用这种直线式程序语言写的程序进行“解释”，为了用“函数式程序设计”风格来编写该函数（这种风格不使用赋值语句），要在声明局部变量的同时对它进行初始化。

In [40]:
type OptBoxedTable = Option<Box<Table>>;

struct Table {
    id: String,
    value: i32,
    tail: OptBoxedTable,
}

impl Table {
    fn lookup(t: OptBoxedTable, id: &str) -> (Option<i32>, OptBoxedTable) {
        if let Some(t) = t {
            if t.id == id {
                return (Some(t.value), Some(t));
            }
            if let Some(tail) = t.tail {
                return Self::lookup(Some(tail), id);
            }
            (None, Some(t))
        } else {
            (None, t)
        }
    }

    fn update(tail: OptBoxedTable, id: String, value: i32) -> OptBoxedTable {
        Some(Box::new(Self { id, value, tail }))
    }
}

type IntAndTable = (i32, OptBoxedTable);

fn inter_exp(exp: BoxedExp, t: OptBoxedTable) -> IntAndTable {
    match *exp {
        Exp::Id(id) => {
            let (v, t) = Table::lookup(t, id.as_ref());
            (
                v.expect(format!("Error: {} is not defined.", id).as_ref()),
                t,
            )
        }
        Exp::Num(v) => (v, t),
        Exp::Op { left, op, right } => {
            let (lv, t) = inter_exp(left, t);
            let (rv, t) = inter_exp(right, t);
            let v = match op {
                BinOp::Plus => lv + rv,
                BinOp::Minus => lv - rv,
                BinOp::Times => lv * rv,
                BinOp::Div => lv / rv,
            };
            (v, t)
        }
        Exp::Eseq(stm, exp) => inter_exp(exp, interp_stm(stm, t)),
    }
}

fn print_exp_list(expList: BoxedExpList, t: OptBoxedTable) -> OptBoxedTable {
    match *expList {
        ExpList::Pair { head, tail } => {
            let (v, t) = inter_exp(head, t);
            print!("{} ", v);
            print_exp_list(tail, t)
        }
        ExpList::Last(exp) => {
            let (v, t) = inter_exp(exp, t);
            println!("{}", v);
            t
        }
    }
}

fn interp_stm(stm: BoxedStm, t: OptBoxedTable) -> OptBoxedTable {
    match *stm {
        Stm::Compound(stm1, stm2) => interp_stm(stm2, interp_stm(stm1, t)),
        Stm::Assign { id, exp } => {
            let (v, t) = inter_exp(exp, t);
            Table::update(t, id, v)
        }
        Stm::Print(expList) => print_exp_list(expList, t),
    }
}


interp_stm(prog, None);

8 7
80
